<a href="https://colab.research.google.com/github/AmitNikhade/AmitNikhade/blob/main/Finetuning_T5_7-6-2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Few shot text generation with T5 Transformer**

Author: Ramsri Goutham Golla

Linkedin : https://www.linkedin.com/in/ramsrig/

Twitter: https://twitter.com/ramsri_goutham

## 1. Install libraries

In [1]:
!pip install transformers==2.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 635 kB 5.0 MB/s 
     |████████████████████████████████| 5.6 MB 50.8 MB/s 
     |████████████████████████████████| 1.2 MB 48.4 MB/s 
     |████████████████████████████████| 880 kB 59.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d3cd2e14482f02f553b8187f4c5768fb9b4e5ae08eeb3d76fcefd595347f53b5
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [2]:
# Check we have a GPU and check the memory size of the GUP
!nvidia-smi

Tue Jun  7 11:16:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2. Prepare Model

In [1]:

import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')


In [3]:
# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in t5_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in t5_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=1e-8)
t5_model.to('cuda')


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [31]:
# df.columns

In [41]:
import pandas as pd
df = pd.read_csv("/content/file1.csv")
# df
df = df.drop(columns=['answers'])
df = df.drop(columns=['Unnamed: 0'])
# # # df
df['answers'] = [[str(i),j] for i,j in zip(df['answer_start'],df['answer_text'])]
dp = df.drop(columns = ['id', 'context', 'answer_start', 'answers'])
o = []
u = 0
for i,j in dp.iterrows():
  # print(j['question'],j['answer_text'])
  o.append((j['question'],j['answer_text']))
  u = u+1
  if u == 50:
    break
  # break
# o

In [42]:
# dataset preparation

# true_false_adjective_tuples = [
#                                ("The cat is alive","The cat is dead"),
#                                ("The old woman is beautiful","The old woman is ugly"),
#                                ("The purse is cheap","The purse is expensive"),
#                                ("Her hair is curly","Her hair is straight"),
#                                ("The bathroom is clean","The bathroom is dirty"),
#                                ("The exam was easy","The exam was difficult"),
#                                ("The house is big","The house is small"),
#                                ("The house owner is good","The house owner is bad"),
#                                ("The little kid is fat","The little kid is thin"),
#                                ("She arrived early","She arrived late."),
#                                ("John is very hardworking","John is very lazy"),
#                                ("The fridge is empty","The fridge is full")

# ]
true_false_adjective_tuples = o

In [46]:
true_false_adjective_tuples

[('In what year was Ronald Adair murdered?', '1894'),
 ('Who was murdered in the spring of 1894?', 'Ronald Adair'),
 ('Why was the case suppressed?',
  'the prosecution was so overwhelmingly strong that it was not necessary to bring forward all the facts'),
 ('How long have I been able to supply the missing links that make up the whole remarkable chain?',
  'nearly ten years'),
 ('What was the greatest shock and surprise of any event in my adventurous life?',
  'the inconceivable sequel'),
 ('What did the sequel give me?',
  'the greatest shock and surprise of any event in my adventurous life'),
 ('What kind of emotions do I feel after this long interval?',
  'joy, amazement, and incredulity'),
 ('What happened to the positive prohibition from his lips?', 'withdrawn'),
 ("How did I feel about Sherlock Holmes's methods?", 'indifferent success'),
 ('Who did I have a close relationship with?', 'Sherlock Holmes'),
 ('Whose tragedy appealed to me most?', 'Ronald Adair'),
 ('Whose death caus

## 3. Train Loop

In [45]:
t5_model.train()

epochs = 50
device = 'cuda'
for epoch in range(epochs):
  print ("epoch ",epoch)
  for input,output in true_false_adjective_tuples:
    input_sent = "Question: "+input+ " </s>"
    ouput_sent = output+" </s>"

    tokenized_inp = tokenizer.encode_plus(input_sent,  max_length=96, pad_to_max_length=True,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(ouput_sent, max_length=96, pad_to_max_length=True,return_tensors="pt")


    input_ids  = tokenized_inp["input_ids"].to(device, dtype=torch.long)
    attention_mask = tokenized_inp["attention_mask"].to(device, dtype=torch.long)

    lm_labels= tokenized_output["input_ids"].to(device, dtype=torch.long)
    decoder_attention_mask=  tokenized_output["attention_mask"].to(device, dtype=torch.long)


    # the forward function automatically creates the correct decoder_input_ids
    output = t5_model(input_ids=input_ids, lm_labels=lm_labels,decoder_attention_mask=decoder_attention_mask,attention_mask=attention_mask)
    loss = output[0]

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()




epoch  0
epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9
epoch  10
epoch  11
epoch  12
epoch  13
epoch  14
epoch  15
epoch  16
epoch  17
epoch  18
epoch  19
epoch  20
epoch  21
epoch  22
epoch  23
epoch  24
epoch  25
epoch  26
epoch  27
epoch  28
epoch  29
epoch  30
epoch  31
epoch  32
epoch  33
epoch  34
epoch  35
epoch  36
epoch  37
epoch  38
epoch  39
epoch  40
epoch  41
epoch  42
epoch  43
epoch  44
epoch  45
epoch  46
epoch  47
epoch  48
epoch  49


## 4. Test model

In [53]:
test_sent = 'Question: hii!. </s>'
test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

test_input_ids  = test_tokenized["input_ids"].to(device, dtype=torch.long)
test_attention_mask = test_tokenized["attention_mask"].to(device, dtype=torch.long)

t5_model.eval()
beam_outputs = t5_model.generate(
    input_ids=test_input_ids,attention_mask=test_attention_mask,
    max_length=64,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=1,
    no_repeat_ngram_size=2
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)

hi


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


In [ ]:
test_sent = 'falsify: This is a safe neighbourhood. </s>'
test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

test_input_ids  = test_tokenized["input_ids"]
test_attention_mask = test_tokenized["attention_mask"]

t5_model.eval()
beam_outputs = t5_model.generate(
    input_ids=test_input_ids,attention_mask=test_attention_mask,
    max_length=64,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=3,
    no_repeat_ngram_size=2
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)

This is a safe neighbourhood.
This is a safe neighbourhood
This is a dangerous neighbourhood.


In [ ]:
test_sent = 'falsify: The tortoise was very slow. </s>'
test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

test_input_ids  = test_tokenized["input_ids"]
test_attention_mask = test_tokenized["attention_mask"]

t5_model.eval()
beam_outputs = t5_model.generate(
    input_ids=test_input_ids,attention_mask=test_attention_mask,
    max_length=64,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=3,
    no_repeat_ngram_size=2
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)

The tortoise was very slow
The tortoise was very fast
The tortoise was very quick
